**Temat:** Analiza sentymentu w tekstach internetowych w oparciu o sieci typu Transformer

**Wprowadzenie:** Analiza sentymentu to technika przetwarzania języka naturalnego (NLP), która identyfikuje ton emocjonalny w tekście, klasyfikując go na pozytywny, negatywny lub neutralny. Wykorzystuje się ją do badania opinii klientów, monitorowania reputacji marki czy analizy treści mediów społecznościowych.

**Cel projektu:** Celem projektu jest opracowanie i implementacja modelu analizy sentymentu, który pozwoli na klasyfikację opinii użytkowników na podstawie tekstów pochodzących z Internetu. Należy przeanalizować dane tekstowe, przygotować odpowiedni model oraz zaprezentować wyniki analizy.

In [14]:
!pip3 install datasets transformers torch 'numpy<2' accelerate --quiet

### Ładowanie danych

In [15]:
from datasets import load_dataset

ds = load_dataset("clapAI/MultiLingualSentiment")

In [180]:
# what languages are available
languages = ds['train'].unique('language')
print("Available languages:", languages)
ds_types = ['train', 'validation', 'test']
# Create dictionary to store datasets for each language
datasets_by_language = {}

# # Split train, validation and test for each language
for lang in languages:
    datasets_by_language[lang] = {}
    for ds_type in ds_types:
        datasets_by_language[lang][ds_type] = ds[ds_type].filter(
            lambda batch: [x == lang for x in batch['language']],
            batched = True,
            num_proc=4
            )
        
        # Reduce dataset by 100 times
        rows_counter = datasets_by_language[lang][ds_type].num_rows
        new_num_rows = round(rows_counter*0.01)
        datasets_by_language[lang][ds_type] = datasets_by_language[lang][ds_type].select(range(new_num_rows))
    

Available languages: ['en', 'es', 'ja', 'ar', 'tr', 'fr', 'vi', 'zh', 'de', 'ru', 'ko', 'id', 'multilingual', 'pt', 'ms', 'hi', 'it']


### Tworzenie testowego datasetu

In [181]:
from datasets import concatenate_datasets

test_languages = ['en', 'es', 'zh']
test_ds_list = [datasets_by_language[lang]['test'] for lang in test_languages]
test_dataset = concatenate_datasets(test_ds_list)
test_dataset = test_dataset.shuffle(seed=42)

## Zero-shot Prompting

In [55]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


model_name = 'Qwen/Qwen3-0.6B'
tokenizer = AutoTokenizer.from_pretrained(model_name)
# For MacBooks with CPU Intel you have to set device_map as cpu and torch_dtype as torch.float32 
# otherwise it doesn't compile
zero_shot_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float32,
    device_map={"": "cpu"})
zero_shot_model.eval()

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layernorm): Qwe

In [124]:
# Prompt template
def build_prompt(text):
    return f"Just define in one word the sentiment of this text as positive, negative or neutral:\n\"{text}\"\nAnswer(positive/negative/neutral):\n"

def predict_sentiment(text):
    prompt = build_prompt(text)
    zero_shot_model_inputs = tokenizer(prompt, return_tensors="pt").to(zero_shot_model.device)
    generated_ids = zero_shot_model.generate(**zero_shot_model_inputs, max_new_tokens=3)
    answer = tokenizer.batch_decode(generated_ids)[0][len(prompt):]
    return answer

In [192]:
print(test_dataset[0]['text'])
print(predict_sentiment(test_dataset[0]['text']))
print(f'\nReal answer:\n{test_dataset[0]['label']}')

I ordered some birth announcement cards for my son, was really pleased with how they turned out everybody said how nice they were. Great discount for first order as well
Answer:
positive

Real answer:
positive


### Test Zero-shot prompting

In [199]:

# Mapping dataset and get estimates from LLM
def process(result, row):
    text = row['text']
    answer = predict_sentiment(text)
    if row['label'] in answer:
        result[row['language']] += 1

# Calculate total and for each language accuracy
def calc_accuracy(result, ds):
    accuracy = {}
    for lang in result:
        lang_total = ds.filter(
            lambda batch: [x == lang for x in batch['language']],
            batched = True,
            num_proc = 4
        ).num_rows
        accuracy[lang] = result[lang]/lang_total
    total_accuracy = sum(result.values()) / ds.num_rows
    accuracy['total'] = total_accuracy
    return accuracy


In [208]:
zero_shot_results = {lang:0 for lang in test_languages}
test_dataset.map(lambda row: process(zero_shot_results, row))
zero_shot_accuracy = calc_accuracy(zero_shot_results, test_dataset)

Map: 100%|██████████| 2186/2186 [27:37<00:00,  1.32 examples/s] 


Dokładność dla zero-shot prompting

In [209]:
print(f"{'Język':<10} {'Dokładność':<10}")
print('-' * 22)
for lang, acc in zero_shot_accuracy.items():
    print(f"{lang:<10} {acc:<10.2f}")

    

Język      Dokładność
----------------------
en         0.91      
es         0.92      
zh         0.91      
total      0.91      
